In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
pip install openmatrix

In [7]:
import os
import sys
import numpy as np
import pandas as pd
import openmatrix as omx

In [ ]:
# Assume you have not changed the relative paths after cloning the repository
root = '/content/drive/MyDrive/Spring 24/Network/CEE520 COS520 Project/TransportationNetworks'
root

'/content/drive/MyDrive/Spring 24/Network/CEE520 COS520 Project/TransportationNetworks'

In [8]:
# Meet
root = '/content/drive/MyDrive/CEE520 COS520 Project/TransportationNetworks'

In [9]:
# We list all folders available, most of which are TNTP instances
folders = [x for x in os.listdir(root)[1:] if os.path.isdir(os.path.join(root, x))]

In [10]:
print(folders)

['Berlin-Prenzlauerberg-Center', 'Berlin-Mitte-Center', 'Barcelona', 'Berlin-Center', 'Austin', 'Berlin-Mitte-Prenzlauerberg-Friedrichshain-Center', 'Anaheim', 'Berlin-Tiergarten', 'Berlin-Friedrichshain', 'Birmingham-England', 'Chicago-Sketch', 'Munich', 'Braess-Example', 'chicago-regional', 'GoldCoast', 'SiouxFalls', 'Eastern-Massachusetts', 'Hessen-Asymmetric', 'Philadelphia', 'Sydney', 'SymmetricaTestCase', 'Winnipeg-Asymmetric', 'Winnipeg', 'Terrassa-Asymmetric', '_scripts']


In [11]:
# Function to import OMX matrices
def import_matrix(matfile):
    f = open(matfile, 'r')
    all_rows = f.read()
    blocks = all_rows.split('Origin')[1:]
    matrix = {}
    for k in range(len(blocks)):
        orig = blocks[k].split('\n')
        dests = orig[1:]
        orig=int(orig[0])

        d = [eval('{'+a.replace(';',',').replace(' ','') +'}') for a in dests]
        destinations = {}
        for i in d:
            destinations = {**destinations, **i}
        matrix[orig] = destinations
    zones = max(matrix.keys())
    mat = np.zeros((zones, zones))
    for i in range(zones):
        for j in range(zones):
            # We map values to a index i-1, as Numpy is base 0
            mat[i, j] = matrix.get(i+1,{}).get(j+1,0)

    index = np.arange(zones) + 1

    myfile = omx.open_file('demand.omx','w')
    myfile['matrix'] = mat
    myfile.create_mapping('taz', index)
    myfile.close()

In [12]:
# If we want to import all matrices in place
for f in folders:
    mod = os.path.join(root, f)
    mod_files = os.listdir(mod)

    for i in mod_files:
      try:
        print(f.upper())
        if 'TRIPS' in i.upper() and i.lower()[-5:]=='.tntp':
            print('    trips')
            source_file = os.path.join(mod, i)
            import_matrix(source_file)
      except:
        print(f'folder {i} failed')

BERLIN-PRENZLAUERBERG-CENTER
BERLIN-PRENZLAUERBERG-CENTER
BERLIN-PRENZLAUERBERG-CENTER
    trips
BERLIN-PRENZLAUERBERG-CENTER
BERLIN-PRENZLAUERBERG-CENTER
BERLIN-MITTE-CENTER
BERLIN-MITTE-CENTER
BERLIN-MITTE-CENTER
    trips
BERLIN-MITTE-CENTER
BERLIN-MITTE-CENTER
BARCELONA
BARCELONA
BARCELONA
BARCELONA
    trips
BERLIN-CENTER
BERLIN-CENTER
BERLIN-CENTER
BERLIN-CENTER
BERLIN-CENTER
    trips
AUSTIN
    trips
AUSTIN
AUSTIN
AUSTIN
BERLIN-MITTE-PRENZLAUERBERG-FRIEDRICHSHAIN-CENTER
    trips
BERLIN-MITTE-PRENZLAUERBERG-FRIEDRICHSHAIN-CENTER
BERLIN-MITTE-PRENZLAUERBERG-FRIEDRICHSHAIN-CENTER
BERLIN-MITTE-PRENZLAUERBERG-FRIEDRICHSHAIN-CENTER
BERLIN-MITTE-PRENZLAUERBERG-FRIEDRICHSHAIN-CENTER
ANAHEIM
ANAHEIM
ANAHEIM
ANAHEIM
ANAHEIM
    trips
ANAHEIM
ANAHEIM
BERLIN-TIERGARTEN
BERLIN-TIERGARTEN
    trips
BERLIN-TIERGARTEN
BERLIN-TIERGARTEN
BERLIN-TIERGARTEN
BERLIN-FRIEDRICHSHAIN
BERLIN-FRIEDRICHSHAIN
BERLIN-FRIEDRICHSHAIN
    trips
BERLIN-FRIEDRICHSHAIN
BERLIN-FRIEDRICHSHAIN
BIRMINGHAM-ENGLAND
  

In [ ]:
# Importing the networks into a Pandas dataframe consists of a single line of code
# but we can also make sure all headers are lower case and without trailing spaces

netfile = os.path.join(root, 'SiouxFalls','SiouxFalls_net.tntp')
net = pd.read_csv(netfile, skiprows=8, sep='\t')

trimmed= [s.strip().lower() for s in net.columns]
net.columns = trimmed

# And drop the silly first andlast columns
net.drop(['~', ';'], axis=1, inplace=True)

In [ ]:
net.head()

,init_node,term_node,capacity,length,free_flow_time,b,power,speed,toll,link_type
0,1,2,25900.200640,6,6,0.15,4,0,0,1
1,1,3,23403.473190,4,4,0.15,4,0,0,1
2,2,1,25900.200640,6,6,0.15,4,0,0,1
3,2,6,4958.180928,5,5,0.15,4,0,0,1
4,3,1,23403.473190,4,4,0.15,4,0,0,1


In [ ]:
net.columns

Index(['init_node', 'term_node', 'capacity', 'length', 'free_flow_time', 'b',
       'power', 'speed', 'toll', 'link_type'],
      dtype='object')

In [17]:
for a in ['Barcelona']:
  netfile = os.path.join(root, a,a +'_net.tntp')
  net = pd.read_csv(netfile, skiprows=8, sep='\t')

  trimmed= [s.strip().lower() for s in net.columns]
  net.columns = trimmed

  # And drop the silly first andlast columns
  net.drop(['~', ';'], axis=1, inplace=True)
  net.to_csv(f'/content/drive/MyDrive/CEE520 COS520 Project/CSV_networks/{a}.csv')

In [ ]:
# Importing the networks into a Pandas dataframe consists of a single line of code
# but we can also make sure all headers are lower case and without trailing spaces

netfile = os.path.join(root, 'SiouxFalls','SiouxFalls_net.tntp')
net = pd.read_csv(netfile, skiprows=8, sep='\t')

trimmed= [s.strip().lower() for s in net.columns]
net.columns = trimmed

# And drop the silly first andlast columns
net.drop(['~', ';'], axis=1, inplace=True)

# TO DO
# 'Berlin-Friedrichshain'
# 'Berlin-Mitte-Center'
#'Eastern-Massachusetts',

In [19]:
netfile = os.path.join(root, 'Berlin-Friedrichshain','friedrichshain-center_net.tntp')
net = pd.read_csv(netfile, skiprows=8, sep='\t')

trimmed= [s.strip().lower() for s in net.columns]
net.columns = trimmed

# And drop the silly first andlast columns
net.drop(['~', ';'], axis=1, inplace=True)
net.to_csv(f'/content/drive/MyDrive/CEE520 COS520 Project/CSV_networks/Berlin-Friedrichshain.csv')

netfile = os.path.join(root, 'Berlin-Mitte-Center','berlin-mitte-center_net.tntp')
net = pd.read_csv(netfile, skiprows=8, sep='\t')

trimmed= [s.strip().lower() for s in net.columns]
net.columns = trimmed

# And drop the silly first andlast columns
net.drop(['~', ';'], axis=1, inplace=True)
net.to_csv(f'/content/drive/MyDrive/CEE520 COS520 Project/CSV_networks/Berlin-Mitte-Center.csv')


netfile = os.path.join(root, 'Eastern-Massachusetts','EMA_net.tntp')
net = pd.read_csv(netfile, skiprows=8, sep='\t')

trimmed= [s.strip().lower() for s in net.columns]
net.columns = trimmed

# And drop the silly first andlast columns
net.drop(['~', ';'], axis=1, inplace=True)
net.to_csv(f'/content/drive/MyDrive/CEE520 COS520 Project/CSV_networks/Eastern-Massachusetts.csv')
